In [53]:
###Rudimentary test suite to ensure that updates do not break the code 
using Ronin
using Missings 
using HDF5 
using NCDatasets
using BenchmarkTools 
using StatsBase
using Scratch

using JLD2 
include("../src/DecisionTree/DecisionTree.jl")
global scratchspace = @get_scratch!("ronin_testing")


###Will undergo a basic training/QC pipeline. Model is not meant to output 
###Correct results, but rather simply show that it can produce data, train a model, 
###and correctly apply the model to a scan. 

###The below will be testing for a single-pass model

TRAINING_PATH = "../BENCHMARKING/benchmark_cfrads/"
config_file_path = "../BENCHMARKING/benchmark_setup/config.txt"
sample_model = "../BENCHMARKING/benchmark_setup/benchmark_model.joblib"

"../BENCHMARKING/benchmark_setup/benchmark_model.joblib"

In [54]:
function write_task_to_file(filepath,tasks)
    isfile(filepath) ? rm(filepath) : ""
    open(filepath, "w") do f
        for task in tasks
            write(f, "$(task),")
        end 
    end 
end 

write_task_to_file (generic function with 1 method)

In [55]:
write_task_to_file("./sample_argfile.txt", ["VEL"])

In [56]:
cfrad_loc = "/Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/NOAA_benchmark_cfrads/cfrad.20220907_125306.750_to_20220907_125310.729_N42RF-TS_AIR.nc"
argfile_path = "./sample_argfile.txt" 
output_file = "placeholder.h5"

"placeholder.h5"

In [57]:
using StatsPlots
languages = repeat(["Julia", "Python"], inner =5)
params = ["DBZ", "AVG(DBZ)", "STD(DBZ)", "ISO(DBZ)", "AHT"]
nam = repeat(params, outer=2)
ts =[.026 0.0528; .084 3.05; .096 6.28; .087 1.3; .102 122. ]

5×2 Matrix{Float64}:
 0.026    0.0528
 0.084    3.05
 0.096    6.28
 0.087    1.3
 0.102  122.0

In [58]:
using StatsPlots
languages = repeat(["Julia", "Python"], inner =5)
params = ["DBZ", "AVG(DBZ)", "STD(DBZ)", "ISO(DBZ)", "AHT"]
nam = repeat(params, outer=2)
ts =[.026 0.0528; .084 3.05; .096 6.28; .087 1.3; .102 122. ]
groupedbar(nam, ts, group=languages, ylims = (10^-2, 1500), yscale=:log, yticks=[10^-3, 10^-2, 10^-1, 10^0, 10, 100, 1000], 
            ylabel = "Calculation Time (s)", xlabel = "Parameter", title = "RONIN Input Feature Calculation Time\n Julia vs. Python", dpi=300)
fig = plot!(size=(800,800))
savefig(fig, "Comparison_new.png")

"/Users/ischluesche/Documents/Grad_School/Research/Ronin/test/Comparison_new.png"

In [59]:
####FOR ISS CONFERENCE: TESTING VERSUS THE PYTHON VERSION 

time = @benchmark X, IDXER = calculate_features(cfrad_loc, argfile_path, output_file, false; remove_variable = "VEL", verbose=false,
            REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)

COMPLETED PROCESSING 1 FILES IN 0.57 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32}
COMPLETED PROCESSING 1 FILES IN 0.02 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32}
COMPLETED PROCESSING 1 FILES IN 0.02 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32}
COMPLETED PROCESSING 1 FILES IN 0.02 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32}
COMPLETED PROCESSING 1 FILES IN 0.04 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32}
COMPLETED PROCESSING 1 FILES IN 0.03 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: placeholder.h5

WRITING DATA TO FILE OF SHAPE (101853, 1)
X TYPE: Matrix{Float32

BenchmarkTools.Trial: 203 samples with 1 evaluation per sample.
 Range (min … max):  17.159 ms … 186.228 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     21.704 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.471 ms ±  14.537 ms  ┊ GC (mean ± σ):  4.44% ± 6.16%

    ▆▇▄█▃▆                                                      
  ▆▆██████▇█▄▃▃▃▄▄▃▃▁▁▃▁▁▁▃▁▁▁▃▁▃▁▃▁▁▁▁▁▁▃▃▁▁▁▁▁▁▃▃▁▁▁▃▁▁▁▁▁▁▃ ▃
  17.2 ms         Histogram: frequency by time         65.1 ms <

 Memory estimate: 7.82 MiB, allocs estimate: 5962.

In [60]:


currs = NCDataset(cfrad_loc) 
(newX, newY, newIdx) = process_single_file(currs, argfile_path; 
                                                HAS_INTERACTIVE_QC = false, REMOVE_LOW_NCP = false, NCP_THRESHOLD = 1.f0, 
                                                REMOVE_HIGH_PGG = false, PGG_THRESHOLD=1.f0, QC_variable = "VG", remove_variable = "DBZ", 
                                                replace_missing=false)

(Float32[0.0; 0.0; … ; -32000.0; -32000.0;;], false, Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [154]:
deg2rad

deg2rad (generic function with 6 methods)

In [155]:
sin.(deg2rad.([1. 2. 3.]))

1×3 Matrix{Float64}:
 0.0174524  0.0348995  0.052336

In [61]:
###NEED TO ALLOW THIS TO IGNORE COMMENTS 
tasks = Ronin.get_task_params(config_file_path)

placeholder_matrix = allowmissing(ones(3,3))
center_weight::Float64 = 0

###Weight matrixes for calculating spatial parameters 
iso_weights::Matrix{Union{Missing, Float64}} = allowmissing(ones(7,7))
iso_weights[4,4] = center_weight 
iso_window::Tuple{Int64, Int64} = (7,7)

avg_weights::Matrix{Union{Missing, Float64}} = allowmissing(ones(5,5))
avg_weights[3,3] = center_weight 
avg_window::Tuple{Int64, Int64} = (5,5)

std_weights::Matrix{Union{Missing, Float64}} = allowmissing(ones(5,5))
std_weights[3,3] = center_weight 
std_window::Tuple{Int64, Int64} = (5,5)


weight_matrixes = [placeholder_matrix, placeholder_matrix, std_weights, placeholder_matrix, placeholder_matrix, iso_weights]

path1 = joinpath(scratchspace, "_1.h5")
path2 = joinpath(scratchspace, "_2.h5")

        
function clean_config() 

    task_path = "./tasks.txt"


    task_paths = [task_path, task_path, task_path] 
    input_path = ds_path
    num_models = 3
    initial_met_prob = (.1, .9) 
    final_met_prob = (.1,.9)
    
    ###Combine into vector for model configuration object 
    ###It's important to note that len(met_probs) is enforced to be equal to num_models 
    met_probs = [initial_met_prob, initial_met_prob, final_met_prob]
    
    ###The following are default windows specified in RoninConstants.jl 
    ###Standard 7x7 window 
    sw = Ronin.standard_window 
    ###7x7 window with only nonzero weights in azimuth dimension 
    aw = Ronin.azi_window
    ###7x7 window with only nonzero weights in range dimension 
    rw = Ronin.range_window 
    ###Placeholder window for tasks that do not require spatial context 
    pw = Ronin.placeholder_window 
    
    ###Specify a weight matrix for each individual task in the configuration file 
    weight_vec = [pw, rw]
    ###Specify a weight vector for each model pass 
    ###len(weight_vector) is enforced to be equal to num_models (should have a set of weights for each pass) 
    task_weights = [weight_vec, weight_vec, weight_vec] 
    
    base_name = joinpath(scratchspace, "raw_model")
    base_name_features = joinpath(scratchspace, "output_features")
    ###List of paths to output trained models to. Enforced to be same size as num_models 
    model_output_paths = [base_name * "_$(i-1).jld2" for i in 1:num_models ]
    ###List of paths to output calculated features to. Enforced to be same size as num_models 
    feature_output_paths = [base_name_features * "_$(i-1).h5" for i in 1:num_models]
    
    
    ###Options are "balanced" or "". If "balanced", the decision trees will be trained 
    ###on a weighted version of the existing classes in order to combat class imbalance 
    class_weights = "balanced"
    
    ###Name of variable in cfradials that has already had interactive QC applied 
    QC_var = "VG"
    
    ###Name of a variable in cfradials that will be used to mask what gates are predicted upon.
    ###Missing values in this variable mean that gates will be removed
    remove_var = "VV"
    ###Name of a variable in input cfradials that has not had postprocessing applied. 
    ###This variable is used to determine where MISSING gates exist in the scan 
    remove_var = "VEL"
    
    ###Whether or not the input features for the model have already been calculated 
    file_preprocessed = [false, false, false]
    
    ###Where to write out the masks to in cfradial file. 
    mask_names = ["PASS_1_MASK", "PASS_2_MASK", "PASS_3_MASK"]
    
    
    ###Create model config object
    config = ModelConfig(num_models = num_models,model_output_paths =  model_output_paths,met_probs =  met_probs, 
                        feature_output_paths = feature_output_paths, input_path = input_path,task_mode="nan",file_preprocessed = file_preprocessed,
                         task_paths = task_paths, QC_var = QC_var, remove_var = remove_var, QC_mask = false, mask_names = mask_names,
                         VARS_TO_QC = ["VEL"], class_weights = class_weights, HAS_INTERACTIVE_QC=true, task_weights = task_weights,
                         REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)

end

clean_config (generic function with 1 method)

In [170]:
##Create a toy sized cfradial file that we know the exact values of calculations for 
ds_path = joinpath(scratchspace, "toy_set.nc")
isfile(ds_path) && rm(ds_path)
ds = NCDataset(ds_path, "c")
# import Base.size
# import Base.length 
# function size(v::Matrix{Union{Missing, Float32}}, dim::Int64)
    
#     return 5
# end 

# function length(v::Nothing)
#     return 5
# end 
####start with 5x5 
range_dim = 5
time_dim  = 5

times = collect(1:1:5)
ranges = collect(1:5000:25000)
alt_step = 5
alts = collect(2000:5:2000 + (length(ranges) - 1) * alt_step)

elev_angle_step = -15
elevation_angles = collect(5:elev_angle_step:5 + (length(ranges) - 1) * elev_angle_step)

az_angle_step = 15 
az_angles = collect(5:az_angle_step:5+(length(ranges) -1) * az_angle_step)

sample_DBZ = Matrix{Union{Missing, Float32}}(reshape(sample(1:65, range_dim*time_dim),(range_dim, time_dim)))
sample_VEL = Matrix{Float32}(reshape(sample(-20:20, range_dim*time_dim), (range_dim, time_dim)))
sample_NCP = fill(1., (range_dim, time_dim))
sample_NCP[:,1] .= .1

sample_PGG = fill(.1, (range_dim, time_dim))
sample_PGG[1,:] .= 1

sample_VG = Matrix{Union{Missing, Float32}}(sample_VEL)
###Collocate with the low values of NCP 
sample_VG[:,1] .= missing
###Add center ro 
sample_VG[3,:] .= missing 


defDim(ds, "range", range_dim)
defDim(ds, "time", time_dim)


tv = defVar(ds, "time", Float32, ("time",), attrib=Dict("units" => "s"))
tv[:] = times

rv = defVar(ds, "range", Float32, ("range",), attrib=Dict("units" => "m"))
rv[:] = ranges

altitude = defVar(ds, "altitude", Float32, ("time", ), attrib=Dict("units" => "m"))
altitude[:] = alts 

ev = defVar(ds, "elevation", Float32, ("time",), attrib=Dict("units" => "degrees"))
ev[:] = elevation_angles

az = defVar(ds, "azimuth", Float32, ("time",), attrib=Dict("units" => "degrees"))
az[:] = az_angles

NCP = defVar(ds, "NCP", Float32, ("range", "time"), attrib=Dict("units" => "NCP units"))
NCP[:,:] = sample_NCP 



VEL = defVar(ds, "VEL", Float32, ("range", "time"), attrib=Dict("units" => "m/s"))
VEL[:,:] = sample_VEL


defVar(ds, "DBZ", sample_DBZ, ("range", "time"), attrib=Dict("units" => "log"))
defVar(ds, "PGG", sample_PGG, ("range", "time"), attrib=Dict("units" => "percent"))
defVar(ds, "VG", sample_VG, ("range", "time"), attrib=Dict("units" => "m/s"))

close(ds)

closed Dataset

In [ ]:
sample_altitudes = []

In [131]:
##Set up a toy example to do the analytical calculations 

function slide_window(var::Matrix{Union{Missing, Float32}}, window::Matrix{Union{Missing,Float32}}, func; ignore_missing = true)

    ##First, construct matrix and apply window 
    nrow, ncol = size(var)
    wrow, wcol = size(window) 

    ###int divide to get the windows off center 
    plusx = div(wcol, 2)
    plusy = div(wrow, 2)

    ###Get center coordinates of window 
    wcx, wcy = (plusx+1, plusy+1)

    res = zeros(size(var))
    curr_view = fill(Missing, (size(window)))

    ###iterate through each position in the input array 
    for i in 1:1:nrow
        for j in 1:1:ncol
            
            ###Calculate maximum and minimum indicies into the variable array 
            
            ###Current row (i) plus the maximum y offset, if this extends outside of bounds, just take maximum row value 
            maxx = i + plusy > nrow ? nrow : i + plusy 
            ###Current column (j) plus the maximum x (horizontal) offset, if this extends outside of bounds, just take maximum column value 
            maxy = j + plusx > ncol ? ncol : j + plusx
            ###Current row (i) minus maximum y (vertical) offset, if this extends outside of bounds, take minimum row value (1) 
            minx = i - plusy < 1    ? 1    : i - plusy 
            ###current column (j) minus maximum x (horizontal) offset, if this extends out of bounds, take minimum column value(1)  
            miny = j - plusx < 1    ? 1    : j - plusx 
            

            ###Calculate indicies into the window/weight array 
            ###Difference here is that these will simply be in relation to the center of the window 

            ###Left side of window will be the center minus the difference of the current col and the minimum col 
            ###Right side of the window will be the center plus the difference of the maximum col and the current col 
            windx = ((wcx - (j-miny)), (wcx + (maxy-j)))
            ###Top of window will be center minus difference between current ro
            windy = ((wcy - (i-minx)), (wcy + (maxx-i)))
            ##Calculate resultant matrix    
            if ignore_missing
                curr_window = window[windy[1]:windy[2], windx[1]:windx[2]]
                idxer = .!ismissing.(curr_window)
                weighted = var[minx:maxx, miny:maxy][idxer] .* curr_window[idxer]
            else 
                weighted = var[minx:maxx, miny:maxy] .* window[windy[1]:windy[2], windx[1]:windx[2]]
            end
            #println(weighted)

            # ##i,j is the center index 
            # println([i,j])

            # printstyled("ROW WINDOW: $(minx : maxx) COL WINDOW: $(miny : maxy)\n", color=:green)
            # printstyled("WINDOW INDEX: $(windx) $(windy)\n", color=:blue)
            # if (i,j) == (5,4) 
            #     printstyled("ROW WINDOW: $(minx : maxx) COL WINDOW: $(miny : maxy)\n", color=:green)
            #     printstyled("WINDOW INDEX: $(windx) $(windy)\n", color=:blue)
            # end 

            res[i,j] = func(weighted)
        end 
    end 
    Matrix{Float32}(res)
end 

slide_window (generic function with 1 method)

In [ ]:
###Test development 4/26/25 


function test_airborne_ht

In [63]:
sample_DBZ

5×5 Matrix{Union{Missing, Float32}}:
 12.0   5.0  57.0   6.0  26.0
 30.0  31.0  46.0  28.0  12.0
 34.0   5.0  46.0  39.0  29.0
 35.0  17.0  48.0  58.0  28.0
 40.0  38.0  60.0  17.0   1.0

In [270]:
###Test both versions here 

function test_calc_std()

    new_weights = Matrix{Union{Float32, Missing}}([1. 2. 3.; 4. 99999. 5.; missing 4. 5.])

    @assert round.(Ronin.calc_std(sample_DBZ); digits=3) == round.(slide_window(sample_DBZ, Ronin.std_weights, std); digits=3)
    @assert round.(Ronin.calc_std(sample_DBZ; weights=new_weights, window = size(new_weights)), digits=3) == round.(slide_window(sample_DBZ, new_weights, Ronin.missing_std); digits=3)


end 

function test_calc_avg() 

    new_weights = Matrix{Union{Float32, Missing}}([1. 2. 4.; 4. 9999. 5.; missing 4. 5.])

    @assert round.(Ronin.calc_avg(sample_DBZ); digits=3) == round.(slide_window(sample_DBZ, Ronin.std_weights, Ronin.missing_avg); digits=3)
    @assert round.(Ronin.calc_avg(sample_DBZ; weights=new_weights, window = size(new_weights)), digits=3) == round.(slide_window(sample_DBZ, new_weights, Ronin.missing_avg); digits=3)

end 



function test_calc_pgg() 

    ###Test that this works on the toy set - mainly looking at dimensions and everything 
    pggs = NCDataset(ds_path) do f
        Ronin.calc_pgg(f)
    end 
    ###Ensure correct size 
    @assert size(pggs) == (length(ranges), length(times))
    ###Ensure correct range of values 
    @assert !any(x -> x > 1.0, pggs) 
    @assert !any(x -> x <0.0, pggs)

    ###It's also known how the input arrays are structured
end 

function test_calc_aht() 

    ahts = NCDataset(ds_path) do f
        Ronin.calc_aht(f)
    end 
    ###Ensure correct size 
    @assert size(ahts) == (length(ranges), length(times))
end 

function test_get_num_tasks() 

    fname = "sample_tasks.txt"
    fpath = joinpath(scratchspace, fname) 
    isfile(fpath) && rm(fpath)
    open(fpath, "w") do f 
        write(f, "ONE, TWO, THREE,   , ")
    end 
    ###Write a bunch of stuff to a taskfile 

    @assert get_num_tasks(fpath) == 4

    isfile(fpath) && rm(fpath)
    open(fpath, "w") do f 
        write(f, "ONE, TWO, THREE, ,\n#ONE, TWO, THREE,FOUR")
    end 
    @assert get_num_tasks(fpath) == 3

end 

function test_parse_directory() 

    dirname = "sample_dir" 
    dpath = joinpath(scratchspace, dirname)
    isdir(dpath) && rm(dpath, recursive=true) 
    mkdir(dpath)
    ##Write some sample files to the path 
    sample_file_names =["cfrad_test_sweep_one.nc", "cfrad_test_sweep_two.nc", "error_file"]
    for file in sample_file_names 
        open(joinpath(dpath, file), "w") do f
            write("ok", f) 
        end 
    end 


    res = parse_directory(dpath)
    @assert res == [joinpath(dpath, sample_file_names[1]), joinpath(dpath, sample_file_names[2])]

end 

function test_get_task_params()

    ###Create sample task file 
    fname = "sample_tasks.txt" 
    fpath = joinpath(scratchspace, fname) 
    isfile(fpath) && rm(fpath) 
    ###Expected use case with no empties and only valid tasks 
    open(fpath, "w") do f 
        write(f, "STD(DBZ), DBZ, VEL, NCP, AHT, PGG, AVG(DBZ)")
    end 

    res = get_task_params(fpath, ["VEL", "DBZ", "NCP"])
    @assert res == ["STD(DBZ)", "DBZ", "VEL", "NCP", "AHT", "PGG", "AVG(DBZ)"]
    rm(fpath) 
    open(fpath, "w") do f
        write(f, "VVE(DBZ)") 
    end 
    res = get_task_params(fpath, ["DBZ"])
    ###Test one with an unknown function 
    @assert res == String[]

    rm(fpath) 
    open(fpath, "w") do f
        write(f, "#This line is a comment\nSTD(DBZ), DBZ") 
    end 
    res = get_task_params(fpath, ["DBZ"]) 
    @assert res == ["STD(DBZ)", "DBZ"]


end 

test_get_task_params (generic function with 1 method)

In [273]:
taasks = readlines(joinpath(scratchspace, "sample_tasks.txt"))

2-element Vector{String}:
 "#This line is a comment"
 "STD(DBZ), DBZ"

In [274]:
for itm in taasks
    print(itm[1])
end 

#S

In [271]:
res = test_get_task_params()

ERROR: CANNOT CALCULATE VVE of DBZ
Potentially invalid function or missing variable

"#This line is a comment" NOT FOUND IN CFRAD FILE.... POTENTIAL ERROR IN CONFIG FILE


In [267]:
res == String[]

true

In [251]:
Ronin.RADAR_FILE_PREFIX

"cfrad"

In [252]:
test_parse_directory()

ERROR: POTENTIALLY INVALID FILE FORMAT FOR FILE: error_file


In [224]:
test_get_num_tasks()

LoadError: AssertionError: get_num_tasks(fpath) == 3

In [203]:
scratchspace

"/Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing"

In [202]:
test_calc_pgg()
test_calc_std()
test_calc_avg()
test_calc_aht()

5×5 Matrix{Float32}:
 2.0       2.00451    2.00951     2.01402    2.01902
 2.4375    1.1383    -0.101816   -1.19838   -2.07619
 2.87939   0.275996  -2.21021    -4.40883   -6.16994
 3.32471  -0.582891  -4.3152     -7.61732  -10.2622
 3.77441  -1.43787   -6.41725   -10.8229   -14.3535

In [182]:
Ronin.calc_std(sample_DBZ)

5×5 Matrix{Float32}:
 23.3922  23.7313  24.2206  22.5268  23.4947
 21.4836  22.538   21.8668  22.2441  21.1223
 19.3969  20.1706  18.6594  20.0575  20.0926
 20.6434  18.6472  17.4883  17.0132  15.6728
 21.8804  20.264   17.7911  17.8368  17.5744

In [183]:
slide_window(sample_DBZ, Ronin.std_weights, std)

5×5 Matrix{Float32}:
 23.3922  23.7313  24.2206  22.5268  23.4947
 21.4836  22.538   21.8668  22.2441  21.1223
 19.3969  20.1706  18.6594  20.0576  20.0926
 20.6434  18.6472  17.4883  17.0132  15.6728
 21.8804  20.264   17.7911  17.8368  17.5744

5×5 Matrix{Float64}:
 0.0  0.0         0.0  0.0  0.0
 0.0  0.00524302  1.0  1.0  1.0
 0.0  0.547352    1.0  1.0  1.0
 0.0  1.0         1.0  1.0  1.0
 0.0  1.0         1.0  1.0  1.0

In [151]:
ds_path

"/Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc"

In [149]:
test_calc_std()

In [118]:
size([1. 2. ; 1. 2.])

(2, 2)

In [115]:
round.(rstd;digits=2) == round.(astd;digits=2)

true

In [108]:
round

20.553452f0

In [107]:
rstd[1,1]

20.553453f0

In [38]:
rng, nrg = NCDataset(ds_path) do f
    rng = Ronin.calc_rng(f)
    nrg = Ronin.calc_nrg(f)
    return (rng, nrg)
end 

(Float32[1.0 1.0 … 1.0 1.0; 2.0 2.0 … 2.0 2.0; … ; 4.0 4.0 … 4.0 4.0; 5.0 5.0 … 5.0 5.0], Float32[0.0005 0.0004987531 … 0.00049627793 0.0004950495; 0.001 0.0009975062 … 0.0009925559 0.000990099; … ; 0.002 0.0019950124 … 0.0019851117 0.001980198; 0.0025 0.0024937657 … 0.0024813896 0.0024752475])

5×5 Matrix{Float32}:
 1.0  1.0  1.0  1.0  1.0
 2.0  2.0  2.0  2.0  2.0
 3.0  3.0  3.0  3.0  3.0
 4.0  4.0  4.0  4.0  4.0
 5.0  5.0  5.0  5.0  5.0

In [43]:
nrg

5×5 Matrix{Float32}:
 0.0005  0.000498753  0.000497512  0.000496278  0.00049505
 0.001   0.000997506  0.000995025  0.000992556  0.000990099
 0.0015  0.00149626   0.00149254   0.00148883   0.00148515
 0.002   0.00199501   0.00199005   0.00198511   0.0019802
 0.0025  0.00249377   0.00248756   0.00248139   0.00247525

In [39]:
nrg_ans = Matrix{Float32}([0.0005  0.000498753  0.000497512  0.000496278  0.00049505;
0.001   0.000997506  0.000995025  0.000992556  0.000990099;
0.0015  0.00149626   0.00149254   0.00148883   0.00148515;
0.002   0.00199501   0.00199005   0.00198511   0.0019802;
0.0025  0.00249377   0.00248756   0.00248139   0.00247525])

5×5 Matrix{Float32}:
 0.0005  0.000498753  0.000497512  0.000496278  0.00049505
 0.001   0.000997506  0.000995025  0.000992556  0.000990099
 0.0015  0.00149626   0.00149254   0.00148883   0.00148515
 0.002   0.00199501   0.00199005   0.00198511   0.0019802
 0.0025  0.00249377   0.00248756   0.00248139   0.00247525

In [71]:

alt_map = repeat(transpose(alts),length(ranges), 1)
rng_map = repeat(ranges, 1, length(times))

NCDataset(ds_path) do ds 
    @assert Ronin.calc_ncp(ds) == Vector{Float32}(sample_NCP[:])
    @assert Ronin.calc_rng(ds) == rng_map
    @assert Ronin.calc_nrg(ds) == Matrix{Float32}(rng_map ./ alt_map)
end 

In [67]:
thing = Matrix{Float32}(rng_map ./ alt_map)

5×5 Matrix{Float32}:
 0.0005  0.000498753  0.000497512  0.000496278  0.00049505
 0.001   0.000997506  0.000995025  0.000992556  0.000990099
 0.0015  0.00149626   0.00149254   0.00148883   0.00148515
 0.002   0.00199501   0.00199005   0.00198511   0.0019802
 0.0025  0.00249377   0.00248756   0.00248139   0.00247525

In [73]:
####test prob_groundgate 

Ronin.prob_groundgate(0, 1000, 1000, 5)

1.0

In [64]:
nrg[5,5]

0.0024752475f0

In [ ]:
@assert alt_map

In [55]:
row()

LoadError: UndefVarError: `row` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [46]:
nrg_ans[2,2]

0.000997506f0

In [44]:
nrg .== nrg_ans

5×5 BitMatrix:
 1  0  0  0  0
 1  0  0  0  1
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0

In [151]:
ds = NCDataset("./ok.nc", "c")
close(ds)

closed Dataset

In [157]:
NCDataset("./ok.nc", "r") do f 
    @assert f["Peanis"][:,:] == sample_DBZ
end 

In [172]:
test_attrs = Dict("TEST_ATTR_1"=> 42, "TEST_ATTR_2"=> "43242342423423")

Dict{String, Any} with 2 entries:
  "TEST_ATTR_1" => 42
  "TEST_ATTR_2" => "43242342423423"

In [173]:
write_field("./ok.nc", "psadf", sample_DBZ; attribs=test_attrs)

Dataset: ./ok.nc
Group: /

Dimensions
   range = 5
   time = 5

Variables
  ok   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0

  peanits   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0

  Peanis   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0



Variable (dataset closed)

In [207]:
 
read_attrs = NCDataset("./ok.nc", "r") do currf
  Dict(currf["psadf"].attrib)
end 

Dict{Union{AbstractString, Symbol}, Any} with 3 entries:
  "_FillValue"  => -32000.0
  "TEST_ATTR_1" => 42
  "TEST_ATTR_2" => "43242342423423"

In [199]:
NCDataset("./ok.nc", "r")

Dataset: ./ok.nc
Group: /

Dimensions
   range = 5
   time = 5

Variables
  ok   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0

  peanits   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0

  Peanis   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0

  psadf   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     _FillValue           = -32000.0
     TEST_ATTR_1          = 42
     TEST_ATTR_2          = 43242342423423



In [189]:
NCDataset("./ok.nc") do currf 
    print(currf["psadf"].attrib) 
end 

_FillValue           = -32000.0
TEST_ATTR_1          = 42
TEST_ATTR_2          = 43242342423423


In [120]:
task_path = "./tasks.txt"


task_paths = [task_path, task_path] 
input_path = ds_path
num_models = 2
initial_met_prob = (.1, .9) 
final_met_prob = (.1,.9)

###Combine into vector for model configuration object 
###It's important to note that len(met_probs) is enforced to be equal to num_models 
met_probs = [initial_met_prob, final_met_prob]

###The following are default windows specified in RoninConstants.jl 
###Standard 7x7 window 
sw = Ronin.standard_window 
###7x7 window with only nonzero weights in azimuth dimension 
aw = Ronin.azi_window
###7x7 window with only nonzero weights in range dimension 
rw = Ronin.range_window 
###Placeholder window for tasks that do not require spatial context 
pw = Ronin.placeholder_window 

###Specify a weight matrix for each individual task in the configuration file 
weight_vec = [pw, rw]
###Specify a weight vector for each model pass 
###len(weight_vector) is enforced to be equal to num_models (should have a set of weights for each pass) 
task_weights = [weight_vec, weight_vec] 

base_name = "raw_model"
base_name_features = "output_features" 
###List of paths to output trained models to. Enforced to be same size as num_models 
model_output_paths = [base_name * "_$(i-1).jld2" for i in 1:num_models ]
###List of paths to output calculated features to. Enforced to be same size as num_models 
feature_output_paths = [base_name_features * "_$(i-1).h5" for i in 1:num_models]


###Options are "balanced" or "". If "balanced", the decision trees will be trained 
###on a weighted version of the existing classes in order to combat class imbalance 
class_weights = "balanced"

###Name of variable in cfradials that has already had interactive QC applied 
QC_var = "VG"

###Name of a variable in cfradials that will be used to mask what gates are predicted upon.
###Missing values in this variable mean that gates will be removed
remove_var = "VV"
###Name of a variable in input cfradials that has not had postprocessing applied. 
###This variable is used to determine where MISSING gates exist in the scan 
remove_var = "VEL"

###Whether or not the input features for the model have already been calculated 
file_preprocessed = [false, false]

###Where to write out the masks to in cfradial file. 
mask_names = ["PASS_1_MASK", "PASS_2_MASK"]




2-element Vector{String}:
 "PASS_1_MASK"
 "PASS_2_MASK"

In [238]:
NCDataset(ds_path) do ds 
    @assert Ronin.calc_ncp(ds) == Vector{Float32}(sample_NCP[:])
    @assert Ronin.calc_rng(ds) == repeat(ds["range"][:], 1, length(ds["time"]))
    
end 

In [245]:
Ronin._weighted_func([1 2; 1 2], [3 4; 3 4], std) == std([1*3 2*4; 1*3 2*4])

true

In [259]:
Ronin._weighted_func([1 2; 1 2], [3 4; 3 4], std) == std([1*3 2*4; 1*3 2*4])

v1 = AbstractMatrix{Union{Float32, Missing}}([1 missing; 1 2])
v3 = Matrix{Float32}([1 2; 1 2])
v2 = AbstractMatrix{Union{Missing, Float32}}([3 4; 3 4])
Ronin._weighted_func(v1, v2, Ronin.missing_std) == Float32(std([3,3,8]))

Ronin._weighted_func(v3, v2, Ronin.missing_std) == Float32(std([3 8; 3 8]))

true

In [263]:
###Make sure isolation function is working as expected 


sample_iso_array = [1.f0 missing 1.f0; 1.f0 1.f0 missing; missing missing missing]
expected_iso = [1.0f0 2.0f0 2.0f0; 3.0f0 5.0f0 5.0f0; 2.0f0 4.0f0 3.0f0]
Ronin.calc_iso(sample_iso_array; weights = fill(1.f0, (3,3)), window=(3,3))

3×3 Matrix{Float32}:
 2.0  3.0  4.0
 4.0  5.0  6.0
 6.0  7.0  8.0

In [285]:
sample_iso_array = Matrix{Union{Float32, Missing}}(fill(1.0f0, (3,3)))
sample_iso_weights = Matrix{Union{Missing, Float32}}(fill(missing, (3,3)))
map(ismissing, Ronin.calc_iso(sample_iso_array; weights=sample_iso_weights, window=(3,3))) == fill(true, (3,3))

true

In [274]:
sample_iso_array

3×3 Matrix{Union{Missing, Float32}}:
 1.0        missing  1.0
 1.0       1.0        missing
  missing   missing   missing

In [270]:
msings = map((x) -> Float32(ismissing(x)), sample_iso_array)

3×3 Matrix{Float32}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  1.0  1.0

In [273]:
using ImageFiltering
mapwindow((x) -> Ronin._weighted_func(x, fill(1.f0, (3,3)), sum), msings, (3,3), border=Fill(0.0f0))

3×3 Matrix{Float32}:
 1.0  2.0  2.0
 3.0  5.0  4.0
 2.0  4.0  3.0

In [264]:
sample_iso_array

3×3 Matrix{Union{Missing, Float32}}:
 1.0        missing  1.0
 1.0       1.0        missing
  missing   missing   missing

3×3 Matrix{Float32}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [249]:
Ronin._weighted_func([1 missing; 1 2], [3 4; 3 4], Ronin.missing_std) == std([3, 3, 8])

2.8867513459481287

true

missing_std (generic function with 1 method)

In [215]:
std([3,4])

0.7071067811865476

In [121]:
scratchspace

"/Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing"

In [122]:
config = clean_config()
config.input_path = "../BENCHMARKING/benchmark_NOAA_cfrads"

"../BENCHMARKING/benchmark_NOAA_cfrads"

In [123]:
config.model_output_paths

3-element Vector{String}:
 "/Users/ischluesche/.julia/scrat" ⋯ 45 bytes ⋯ "/ronin_testing/raw_model_0.jld2"
 "/Users/ischluesche/.julia/scrat" ⋯ 45 bytes ⋯ "/ronin_testing/raw_model_1.jld2"
 "/Users/ischluesche/.julia/scrat" ⋯ 45 bytes ⋯ "/ronin_testing/raw_model_2.jld2"

In [ ]:
# config.file_preprocessed = [false, false] 
# train_multi_model(config)
# @assert (Base.time() - mtime(config.feature_output_paths[1])) < 1 
# sleep(2) 
# config.file_preprocessed = [true, true] 
# train_multi_model(config) 
@assert (Base.time() - mtime(config.feature_output_paths[1])) > 2

In [130]:
isfile()

LoadError: ArgumentError: collection of path components must be non-empty

LoadError: UndefVarError: `ls` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [135]:
ret1 = evaluate_model(config)


CALCULATING FEATURES FOR PASS: 1
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222204.187_to_20240705_222208.161_N42RF-TM_AIR.nc in 0.11919379234313965 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222216.825_to_20240705_222220.799_N42RF-TM_AIR.nc in 0.12966704368591309 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222229.464_to_20240705_222233.438_N42RF-TM_AIR.nc in 0.11730694770812988 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222242.102_to_20240705_222246.076_N42RF-TM_AIR.nc in 0.1137080192565918 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222254.752_to_20240705_222258.731_N42RF-TM_AIR.nc in 0.10849785804748535 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222307.390_to_20240705_222311.370_N42RF-TM_AIR.nc in 0.11661314964294434 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222320.034_to_20240705_222324.008_N42RF-

Excessive output truncated after 524291 bytes.


     units                = db
     meta_group           = radar_calibration
     _FillValue           = -9999.0

  r_calib_coupler_forward_loss_h   (1)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  r_calib
    Attributes:
     long_name            = radar_calibration_coupler_forward_loss_h_channel
     units                = db
     meta_group           = radar_calibration
     _FillValue           = -9999.0

  r_calib_coupler_forward_loss_v   (1)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  r_calib
    Attributes:
     long_name            = radar_calibration_coupler_forward_loss_v_channel
     units                = db
     meta_group           = radar_calibration
     _FillValue           = -9999.0

  r_calib_dbz_correction   (1)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  r_calib
    Attributes:
     long_name            = calibrated_radar_dbz_correction
     units                = db
     meta_group           =

Row,met_probs,task_paths,class_weights,n_trees,max_depth,precision,recall,f1,true_positives,false_positives,true_negatives,false_negatives
,Array…,Array…,String,Int64,Int64,Float32,Float32,Float32,Int64,Int64,Int64,Int64
1,"Tuple{Float32, Float32}[(0.1, 0.9), (0.1, 0.9), (0.1, 0.9)]","[""./tasks.txt"", ""./tasks.txt"", ""./tasks.txt""]",balanced,21,14,0.648145,0.818569,0.723456,369754,200726,364716,81954


In [136]:
ret2 = evaluate_model(config; models_trained=true)

Row,met_probs,task_paths,class_weights,n_trees,max_depth,precision,recall,f1,true_positives,false_positives,true_negatives,false_negatives
,Array…,Array…,String,Int64,Int64,Float32,Float32,Float32,Int64,Int64,Int64,Int64
1,"Tuple{Float32, Float32}[(0.1, 0.9), (0.1, 0.9), (0.1, 0.9)]","[""./tasks.txt"", ""./tasks.txt"", ""./tasks.txt""]",balanced,21,14,0.648145,0.818569,0.723456,369754,200726,364716,81954


In [138]:
ret1 == ret2

true

In [132]:
for model_path in config.model_output_paths
    isfile(model_path) ? rm(model_path) : ""
end 
for model_path in config.model_output_paths 
    @assert isfile(model_path) 
end 

LoadError: AssertionError: isfile(model_path)

In [94]:
###Test composite prediction 
config = clean_config()
config.input_path =  "../BENCHMARKING/benchmark_NOAA_cfrads"
train_multi_model(config)
###Trained 3 pass model 




CALCULATING FEATURES FOR PASS: 1
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222204.187_to_20240705_222208.161_N42RF-TM_AIR.nc in 0.2894599437713623 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222216.825_to_20240705_222220.799_N42RF-TM_AIR.nc in 0.11131691932678223 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222229.464_to_20240705_222233.438_N42RF-TM_AIR.nc in 0.10559296607971191 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222242.102_to_20240705_222246.076_N42RF-TM_AIR.nc in 0.11139893531799316 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222254.752_to_20240705_222258.731_N42RF-TM_AIR.nc in 0.1089320182800293 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222307.390_to_20240705_222311.370_N42RF-TM_AIR.nc in 0.1069939136505127 seconds
Processed ../BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222320.034_to_20240705_222324.008_N42RF-TM

Excessive output truncated after 524291 bytes.


     units                = meters
     _FillValue           = -9999.0
     meta_group           = instrument_parameters

  antenna_transition   (360)
    Datatype:    Union{Missing, Int8} (Int8)
    Dimensions:  time
    Attributes:
     long_name            = antenna_is_in_transition_between_sweeps
     units                = 
     _FillValue           = -128
     comment              = 1 if antenna is in transition, 0 otherwise

  georefs_applied   (360)
    Datatype:    Union{Missing, Int8} (Int8)
    Dimensions:  time
    Attributes:
     long_name            = georefs_have_been_applied_to_ray
     units                = 
     _FillValue           = -128
     comment              = 1 if georefs have been applied, 0 otherwise

  n_samples   (360)
    Datatype:    Union{Missing, Int32} (Int32)
    Dimensions:  time
    Attributes:
     long_name            = number_of_samples_used_to_compute_moments
     units                = 
     _FillValue           = -9999
     meta_group     

In [139]:
predictions = [] 
targets = [] 
###Test this by interactively going through the models
###start by opening the first model 
for (i, model) in enumerate(config.model_output_paths)
    currm = load_object(model) 

    currh5 = h5open(config.feature_output_paths[i])
    currfeatures = currh5["X"][:,:]
    currtargets = currh5["Y"][:,:][:]
    close(currh5) 

    push!(predictions, DecisionTree.predict_proba(currm, currfeatures))
    push!(targets, currtargets)

    ###Construct prediction vector  
end 


total_predictions = fill(-1, length(predictions[1][:,2]))
total_targets =  fill(-1, length(predictions[1][:,2]))
idxer = fill(true, length(predictions[1][:,2]))
still_to_predict = fill(true, length(predictions[1][:,2]))

###iteratively construct predictions vector 
###THIS IS ALSO TECHNICALLY NOT TESTING THIS ON A NEW SET OF FEATURES, BUT RATHER ONES THAT 
###HAVE ALREADY BEEN CALCULATED 
for (i, prediction_vec) in enumerate(predictions) 

    ###All the gates are still to be predicted upon 
    cp_mps = predictions[i][:,2]
    cp_metprobs = config.met_probs[i]
    ###Subset of gates from current pass to be predicted upon 
    curr_idx = (cp_mps .< cp_metprobs[1]) .| (cp_mps .> cp_metprobs[2])
    println(sum(curr_idx))
    curr_predictions = cp_mps .> fp_metprobs[2]
    ###We predict where both still_to_predict is 1, and curr_idx is 1 
    ###Still_to_predict will have a value of 1s at all locations 
    ###Just overwrite the next set of predictions too 

    ###At the valid locations in the current idxer, we will write gates 
    total_predictions[still_to_predict] .= curr_predictions 
    still_to_predict[still_to_predict] .= .! curr_idx

   
    # sp_mps = predictions[2][:,1]
    # sp_metprobs = config.met_probs[2] 

    # total_predictions[first_sweep_idx] .= 
end 

###DON'T NEED TO ACTUALLY ITERATIVE CONSTRUCT THE FEATURE VALUES? 


921772
46749
6684


In [141]:
prec, rec, f1, tp, fp, tn, fn, t = evaluate_model(Vector{Bool}(total_predictions), Vector{Bool}(targets[1])) 

(0.64814544f0, 0.81856865f0, 0.72345597f0, 369754, 200726, 364716, 81954, 1017150)

In [143]:
ret_total = evaluate_model(config; models_trained=true) 

Row,met_probs,task_paths,class_weights,n_trees,max_depth,precision,recall,f1,true_positives,false_positives,true_negatives,false_negatives
,Array…,Array…,String,Int64,Int64,Float32,Float32,Float32,Int64,Int64,Int64,Int64
1,"Tuple{Float32, Float32}[(0.1, 0.9), (0.1, 0.9), (0.1, 0.9)]","[""./tasks.txt"", ""./tasks.txt"", ""./tasks.txt""]",balanced,21,14,0.648145,0.818569,0.723456,369754,200726,364716,81954


In [145]:
ret_total[1, "f1"] == f1

true

In [68]:
sum((mps .>= config.met_probs[1][1]) .& (mps .<= config.met_probs[1][2]))

95378

In [67]:
config.met_probs[1][2]

0.9f0

In [48]:
currm

LoadError: UndefVarError: `currm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
###Create model config object
config = ModelConfig(num_models = num_models,model_output_paths =  model_output_paths,met_probs =  met_probs, 
                    feature_output_paths = feature_output_paths, input_path = input_path,task_mode="nan",file_preprocessed = [false, false],
                     task_paths = task_paths, QC_var = QC_var, remove_var = remove_var, QC_mask = false, mask_names = mask_names,
                     VARS_TO_QC = ["VEL"], class_weights = class_weights, HAS_INTERACTIVE_QC=true, task_weights = task_weights,
                     REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)

ModelConfig(2, ["raw_model_0.jld2", "raw_model_1.jld2"], Tuple{Float32, Float32}[(0.1, 0.9), (0.1, 0.9)], ["output_features_0.h5", "output_features_1.h5"], "/Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc", "nan", Bool[0, 0], ["./tasks.txt", "./tasks.txt"], [""], Vector[Matrix{Union{Missing, Float32}}[[1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], Matrix{Union{Missing, Float32}}[[1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]]], true, false, false, true, "VG", "VEL", -32000.0f0, false, true, false, ["PASS_1_MASK", "PASS_2_MASK"], ["VEL"], "_QC", "balanced", 21, 14, false, 0.2f0, 1.0f0)

In [8]:
###Tests to ensure removing high/low NCP gates 
###functions as expected


config.REMOVE_HIGH_PGG = false  
config.REMOVE_LOW_NCP = false  

valid_NCP_gates = sum(sample_NCP .> config.NCP_THRESHOLD)
total_gates = length(sample_DBZ)

config.REMOVE_LOW_NCP = true 

try 
    train_multi_model(config)
catch 

end 

NCDataset(config.feature_output_paths[1]) do f
    @assert size(f["X"][:,:])[1] == valid_NCP_gates
end 

config.REMOVE_LOW_NCP = false 
try 
    train_multi_model(config)
catch 

end 

NCDataset(config.feature_output_paths[1]) do f
    @assert size(f["X"][:,:])[1] == total_gates
end 

# valid_PGG_gates = sum(sample_PGG .< config.PGG_THRESHOLD)

# config.REMOVE_HIGH_PGG = true 

# try 
#     train_multi_model(config)
# catch 

# end 

# NCDataset(config.feature_output_paths[1]) do f
#     print(size(f["X"][:,:]))
#     @assert size(f["X"][:,:])[1] == valid_PGG_gates
# end 






CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 1.1991300582885742 seconds
COMPLETED PROCESSING 1 FILES IN 1.35 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (20, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 2.257 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 20 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.7590851783752441 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 90.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0016109943389892578 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
RESULTANT GATES: 10Dataset: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_s

In [7]:
###Cannot change fill value 
# QC_scan(config)

# currs = NCDataset(ds_path)
# #currs["VEL_QC"].attrib["_FillValue"] == config.FILL_VAL
# #currs["PASS_2_MASK"].attrib["_FillValue"] == config.FILL_VAL 
# close(currs)

# config.FILL_VAL = 123.456 
# QC_scan(config)

# currs = NCDataset(ds_path)
# @assert currs["VEL_QC"].attrib["_FillValue"] == config.FILL_VAL
# #@assert currs["PASS_2_MASK"].attrib["_FillValue"] == config.FILL_VAL 
# close(currs)

In [10]:
###Test to ensure that QC_var is properly passed to the functions 
###Does so by ensuring that the returned target Y array is the same as the 
###specified QC_variable 
VG_map = map(! ismissing, sample_VG)
DBZ_map = map( ! ismissing, sample_DBZ)
@assert DBZ_map != VG_map 
config.QC_var = "VG"
X,Y = calculate_features(config.input_path, config.task_paths[1], config.feature_output_paths[1], config.HAS_INTERACTIVE_QC; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                    write_out = config.write_out)

@assert reshape(Y, (range_dim, time_dim)) == VG_map     

config.QC_var = "DBZ"
X,Y = calculate_features(config.input_path, config.task_paths[1], config.feature_output_paths[1], config.HAS_INTERACTIVE_QC; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                    write_out = config.write_out)

@assert reshape(Y, (range_dim, time_dim)) == DBZ_map

Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.01797008514404297 seconds
COMPLETED PROCESSING 1 FILES IN 0.03 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0039370059967041016 seconds
COMPLETED PROCESSING 1 FILES IN 0.01 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}


In [11]:
### Test to ensure remove_var is properly passed to calculate_features by checking
### That the shape of the feature array changes associated with the variable passed to it
config.remove_var = "VG"
X,Y = calculate_features(config.input_path, config.task_paths[1], config.feature_output_paths[1], config.HAS_INTERACTIVE_QC; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                    write_out = config.write_out)
@assert size(X)[1] == sum(VG_map)

config.remove_var = "DBZ"
X,Y = calculate_features(config.input_path, config.task_paths[1], config.feature_output_paths[1], config.HAS_INTERACTIVE_QC; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                    write_out = config.write_out)

@assert size(X)[1] == sum(DBZ_map)

Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.29209303855895996 seconds
COMPLETED PROCESSING 1 FILES IN 0.3 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (16, 2)
X TYPE: Matrix{Float32}
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0012340545654296875 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}


In [10]:
function train_multi_model_bench(config::ModelConfig)
    ##Quick input sanitation check 
    @assert (length(config.model_output_paths) == length(config.feature_output_paths)
             == length(config.met_probs) == length(config.task_paths) == length(config.task_weights))

    full_start_time = time() 
    ###Iteratively train models and apply QC_scan with the specified probabilites to train a multi-pass model 
    ###pipeline 
    for (i, model_path) in enumerate(config.model_output_paths)
        
        out = config.feature_output_paths[i] 
        currt = config.task_paths[i]
        cw = config.task_weights[i]

        ##If execution proceeds past the first iteration, a composite model is being created, and 
        ##so a further mask will be applied to the features 
        if i > 1
            QC_mask = true 
        else 
            QC_mask = config.QC_mask 
        end 

        QC_mask ? mask_name = config.mask_names[i] : mask_name = ""
        println("MASK NAME: $(mask_name)")
        starttime = time() 
        
        if config.file_preprocessed[i]

            print("Reading input features from file $(out)...\n")
            h5open(out) do f
                X = f["X"][:,:]
                Y = f["Y"][:,:]
            end 

        else
            printstyled("\nCALCULATING FEATURES FOR PASS: $(i)\n", color=:green)

            ###Check to see if the features file already exists, if so, delete it so 
            ###that it may be overwritten 
            if config.write_out & config.overwrite_output
                isfile(out) ? rm(out) : ""
            end 

            X,Y = calculate_features(config.input_path, currt, out, config.HAS_INTERACTIVE_QC; 
                                verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                write_out = config.write_out, QC_mask = QC_mask, mask_name = mask_name, weight_matrixes=cw)
            printstyled("FINISHED CALCULATING FEATURES FOR PASS $(i) in $(round(time() - starttime, digits = 3)) seconds...\n", color=:green)
        end 

        printstyled("\nTRAINING MODEL FOR PASS: $(i)\n", color=:green)
        starttime = time() 

        class_weights = Vector{Float32}([0.0,1.0])
        ##Train model based on these features 
        if config.class_weights != ""

            if lowercase(config.class_weights) != "balanced"
                printstyled("ERROR: UNKNOWN CLASS WEIGHT $(config.class_weights)... \nContinuing with no weighting\n", color=:yellow)
            else 

                class_weights = Vector{Float32}(fill(0,length(Y[:,:][:])))
                weight_dict = compute_balanced_class_weights(Y[:,:][:])
                for class in keys(weight_dict)
                    class_weights[Y[:,:][:] .== class] .= weight_dict[class]
                end 

            end 
        end 
        
        printstyled("\n...TRAINING FOR PASS: $(i) ON $(size(X)[1]) GATES...\n", color=:green)
    
        Ronin.train_model(out, model_path, n_trees = config.n_trees, max_depth = config.max_depth, class_weights = class_weights)

        
        ###If this was the last pass, we don't need to write out a mask, and we're done!
        ###Otherwise, we need to mask out the features we want to apply the model to on the next pass 
        if i < config.num_models

            curr_model = load_object(model_path) 
            curr_metprobs = config.met_probs[i]

            paths = Vector{String}() 
            file_path = config.input_path

            if isdir(file_path) 
                paths = parse_directory(file_path)
            else 
                paths = [file_path]
            end 
                
            for path in paths

                dims = Dataset(path) do f
                    (f.dim["range"], f.dim["time"])
                end 
                
                ###NEED to update this if it's beyond two pass so we can pass it the correct mask
                X, Y, idxer = calculate_features(path, currt, out, true; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP, NCP_THRESHOLD=config.NCP_THRESHOLD,
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG,PGG_THRESHOLD=config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing, return_idxer=true,
                                    write_out = false, QC_mask = QC_mask, mask_name = mask_name, weight_matrixes=cw)
                
                met_probs = DecisionTree.predict_proba(curr_model, X)
                if size(met_probs)[2] < 2
                    throw(DomainError(1, "ERROR: ONLY ONE CLASS IN INPUT DATASET")) 
                end 
                met_probs = met_probs[:, 2]
                valid_idxs = (met_probs .> minimum(curr_metprobs)) .& (met_probs .<= maximum(curr_metprobs))
                print("RESULTANT GATES: $(sum(valid_idxs))")
                ##Create mask field, fill it, and then write out
                new_mask = Matrix{Union{Missing, Float32}}(missings(dims))[:]
               
                ##We only care about gates that have met the base QC thresholds, so first index 
                ##by indexer returned from calculate_features, and then set the gates between
                ##the specified probability levels to valid in the mask. The next model pass will 
                ##thus only be calculated upon these features. 
                idxer = idxer[1][:]
                idxer[idxer] .= Vector{Bool}(valid_idxs)
                new_mask[idxer] .= 1.
                new_mask = reshape(new_mask, dims)
    
                write_field(path, config.mask_names[i+1], new_mask, attribs=Dict("Units" => "Bool", "Description" => "Gates between met prob theresholds"))

            end 
        end   
    end 
    printstyled("\n COMPLETED TRAINING MODEL IN $(round(time() - full_start_time, digits = 3)) seconds...\n", color=:green)   
end 


train_multi_model_bench (generic function with 1 method)

In [11]:
config.QC_mask = true

true

In [12]:
###Ensure that we have the full number of gates 
@assert sum(DBZ_map) == (range_dim * time_dim)
###Then try and mask something out... first we need to write it to file though 
###Can just used the QC'ed stuff 

config.QC_mask = true 
config.mask_names = ["VG", "OK"]

try 
    train_multi_model(config)
catch DomainError 
    println("OK")
    NCDataset(config.feature_output_paths[1]) do f1 
        @assert size(f1["X"][:,:])[1] == sum(.! map(ismissing, sample_VG)) 
    end 
    ###We should get a domain variable because we are removing the non-met 
    ###gates in the first pass 
else 
    @assert false
end 

###Now let's try it without the mask 
config.QC_mask = false 

try 
    train_multi_model(config)
catch DomainError 
    ###possible we're just getting 100% accuracy 
    println("DOMAIN ERROR") 
    NCDataset(config.feature_output_paths[1]) do f1 
        println(size(f1["X"][:,:])[1] )
        @assert size(f1["X"][:,:])[1] == length(sample_DBZ)
    end 
else 
    @assert true
end 
###Check to ensure that it's the full size 





CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0019328594207763672 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (16, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 0.005 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 16 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.00040221214294433594 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 100.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0007100105285644531 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
OK

CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63

In [13]:
config.mask_names = ["OK"]
try
    train_multi_model(config)
catch AssertionError

else 
    @assert false 
end 

try 
    composite_prediction(config) 
catch AssertionError 
else 
    @assert false 
end 

config.mask_names = ["OK", "MASK_2"]
config.QC_mask = false 

false

In [8]:
process_single_file(currs, "./tasks.txt"; NCP_THRESHOLD = Float32(.2), PGG_THRESHOLD=Float32(1.))

LoadError: NetCDF error: [31mVariable 'VV' not found in file ./toy_set.nc[39m (NetCDF error code: -49)

In [12]:
####Test tree depth, n trees, etc. 
config.HAS_INTERACTIVE_QC = true
config.QC_var = "VG"
config.n_trees = 40 
config.max_depth = 20 
train_multi_model(config)
classifier = load_object(config.model_output_paths[1])
@assert classifier.n_trees == config.n_trees 
@assert classifier.max_depth == config.max_depth 




CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0063059329986572266 seconds
COMPLETED PROCESSING 1 FILES IN 0.01 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 0.038 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 25 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.0006389617919921875 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 96.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0022912025451660156 seconds
COMPLETED PROCESSING 1 FILES IN 0.01 SECONDS
RESULTANT GATES: 17Dataset: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testin

In [15]:
config.REMOVE_HIGH_PGG = false  
config.REMOVE_LOW_NCP = false  

valid_NCP_gates = sum(sample_NCP .> config.NCP_THRESHOLD)
total_gates = length(sample_DBZ)

config.REMOVE_LOW_NCP = true 

train_multi_model(config)


CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0016720294952392578 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (20, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 0.007 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 20 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.00024819374084472656 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 95.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0009279251098632812 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
RESULTANT GATES: 16Dataset: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing

In [106]:
sample_DBZ

5×5 Matrix{Float32}:
 32.0  65.0  54.0  18.0  52.0
 44.0  22.0  12.0  35.0  63.0
 39.0  12.0  62.0   6.0  30.0
 10.0  24.0  55.0  59.0  49.0
 44.0  31.0  23.0  17.0  54.0

In [13]:
###Create model config object
###Ensure that the file preprocessed flag works correctly by not 
###Modifying the existing features if it's already been processed 
config = ModelConfig(num_models = num_models,model_output_paths =  model_output_paths,met_probs =  met_probs, 
                    feature_output_paths = feature_output_paths, input_path = input_path,task_mode="nan",file_preprocessed = [false, false],
                     task_paths = task_paths, QC_var = QC_var, remove_var = remove_var, QC_mask = false, mask_names = mask_names,
                     VARS_TO_QC = ["VEL"], class_weights = class_weights, HAS_INTERACTIVE_QC=true, task_weights = task_weights,
                     REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)
sleep(1) 

config.file_preprocessed = [false, false] 
train_multi_model(config)
@assert (Base.time() - mtime(config.feature_output_paths[1])) < 1 
sleep(2) 
config.file_preprocessed = [true, true] 
train_multi_model(config) 
@assert (Base.time() - mtime(config.feature_output_paths[1])) > 2



CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0031919479370117188 seconds
COMPLETED PROCESSING 1 FILES IN 0.01 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 0.014 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 25 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.0005218982696533203 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 88.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0008840560913085938 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
RESULTANT GATES: 16Dataset: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing

In [27]:
###Create model config object
###Ensure that the file preprocessed flag works correctly by not 
###Modifying the existing features if it's already been processed 
isfile(config.feature_output_paths[1]) ? rm(config.feature_output_paths[1]) : ""

config = ModelConfig(num_models = num_models,model_output_paths =  model_output_paths,met_probs =  met_probs, 
                    feature_output_paths = feature_output_paths, input_path = input_path,task_mode="nan",file_preprocessed = [false, false],
                     task_paths = task_paths, QC_var = QC_var, remove_var = remove_var, QC_mask = false, mask_names = mask_names,
                     VARS_TO_QC = ["VEL"], class_weights = class_weights, HAS_INTERACTIVE_QC=true, task_weights = task_weights,
                     REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)

train_multi_model(config)


try
    config = ModelConfig(num_models = num_models,model_output_paths =  model_output_paths,met_probs =  met_probs, 
                    feature_output_paths = feature_output_paths, input_path = input_path,task_mode="nan",file_preprocessed = [false, false],
                     task_paths = task_paths, QC_var = QC_var, remove_var = remove_var, QC_mask = false, mask_names = mask_names,
                     VARS_TO_QC = ["VEL"], class_weights = class_weights, HAS_INTERACTIVE_QC=false, task_weights = task_weights,
                     REMOVE_HIGH_PGG=false, REMOVE_LOW_NCP=false)

    train_multi_model(config)
catch Exception 
    println("GOOD!")
else 
    @assert false 
end 




CALCULATING FEATURES FOR PASS: 1
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.001068115234375 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: output_features_0.h5

WRITING DATA TO FILE OF SHAPE (25, 2)
X TYPE: Matrix{Float32}
FINISHED CALCULATING FEATURES FOR PASS 1 in 0.013 seconds...

TRAINING MODEL FOR PASS: 1

...TRAINING FOR PASS: 1 ON 25 GATES...

Opening HDF5.File: (read-only) output_features_0.h5...
FITTING MODEL
COMPLETED FITTING MODEL IN 0.00019407272338867188 seconds

MODEL VERIFICATION:
ACCURACY ON TRAINING SET: 88.0%

SAVING MODEL TO: raw_model_0.jld2 
Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0008571147918701172 seconds
COMPLETED PROCESSING 1 FILES IN 0.0 SECONDS
RESULTANT GATES: 16Dataset: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy

In [ ]:
###Test to ensure that calculate features does not return a Y vector if HAS_INTERACTIVE_QC
###Is set to false 

In [79]:
i = 1
out = config.feature_output_paths[i] 
currt = config.task_paths[i]
cw = config.task_weights[i]
config.write_out = false
config.HAS_INTERACTIVE_QC = false
config.REMOVE_LOW_NCP = true
##If execution proceeds past the first iteration, a composite model is being created, and 
##so a further mask will be applied to the features 
if i > 1
    QC_mask = true 
else 
    QC_mask = config.QC_mask 
end 

QC_mask ? mask_name = config.mask_names[i] : mask_name = ""
    
X,Y,idxer = calculate_features(config.input_path, currt, out, config.HAS_INTERACTIVE_QC; 
                                    verbose = config.verbose, REMOVE_LOW_NCP = config.REMOVE_LOW_NCP,NCP_THRESHOLD=config.NCP_THRESHOLD, 
                                    REMOVE_HIGH_PGG=config.REMOVE_HIGH_PGG, PGG_THRESHOLD = config.PGG_THRESHOLD, QC_variable = config.QC_var, 
                                    remove_variable = config.remove_var, replace_missing = config.replace_missing,
                                    write_out = config.write_out, QC_mask = QC_mask, mask_name = mask_name, weight_matrixes=cw, return_idxer = true)
@assert Y == [0;;]
@assert sum(idxer[1][:]) == sum(sample_NCP .> config.NCP_THRESHOLD)

Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.0023729801177978516 seconds
COMPLETED PROCESSING 1 FILES IN 0.01 SECONDS


true

20

In [50]:
sum(idxer)

5×5 BitMatrix:
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1

In [40]:
[0 ;]

1-element Vector{Int64}:
 0

In [43]:
Y

1×1 Matrix{Int64}:
 0

In [23]:
isfile(config.feature_output_paths[1]) ? rm(config.feature_output_paths[1]) : ""


In [64]:
QC_mask = Matrix{Union{Missing, Float32}}(fill(1.,(5,5)))
QC_mask[3,:] .= missing

5-element view(::Matrix{Union{Missing, Float32}}, 3, :) with eltype Union{Missing, Float32}:
 missing
 missing
 missing
 missing
 missing

In [15]:
rets = composite_prediction(config, return_probs=true)

LOADING MODELS....
(25, 1)Dataset: ./toy_set.nc
Group: /

Dimensions
   range = 5
   time = 5

Variables
  time   (5)
    Datatype:    Float32 (Float32)
    Dimensions:  time
    Attributes:
     units                = s

  range   (5)
    Datatype:    Float32 (Float32)
    Dimensions:  range
    Attributes:
     units                = m

  NCP   (5 × 5)
    Datatype:    Float32 (Float32)
    Dimensions:  range × time
    Attributes:
     units                = NCP units

  DBZ   (5 × 5)
    Datatype:    Float32 (Float32)
    Dimensions:  range × time
    Attributes:
     units                = dBz

  VEL   (5 × 5)
    Datatype:    Float32 (Float32)
    Dimensions:  range × time
    Attributes:
     units                = m/s

  VG   (5 × 5)
    Datatype:    Union{Missing, Float32} (Float32)
    Dimensions:  range × time
    Attributes:
     units                = m/s
     _FillValue           = 9.96921e36

  PASS_2_MASK   (5 × 5)
    Datatype:    Union{Missing, Float64} (Float64)
    

(Bool[0, 0, 0, 0, 0, 1, 0, 1, 1, 1  …  1, 1, 0, 1, 0, 0, 1, 1, 0, 0], [0; 0; … ; 1; 1;;], [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], [0.3333333333333333, 0.6190476190476191, 0.14285714285714285, 0.14285714285714285, 0.23809523809523808, 1.0, 0.6190476190476191, 0.9523809523809523, 1.0, 0.9523809523809523  …  1.0, 1.0, 0.6190476190476191, 0.9523809523809523, 0.47619047619047616, 0.8095238095238095, 1.0, 1.0, 0.8095238095238095, 0.7142857142857143])

In [17]:
X, y, idxers, probs = rets

(Bool[0, 0, 0, 0, 0, 1, 0, 1, 1, 1  …  1, 1, 0, 1, 0, 0, 1, 1, 0, 0], [0; 0; … ; 1; 1;;], [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], [0.3333333333333333, 0.6190476190476191, 0.14285714285714285, 0.14285714285714285, 0.23809523809523808, 1.0, 0.6190476190476191, 0.9523809523809523, 1.0, 0.9523809523809523  …  1.0, 1.0, 0.6190476190476191, 0.9523809523809523, 0.47619047619047616, 0.8095238095238095, 1.0, 1.0, 0.8095238095238095, 0.7142857142857143])

In [19]:
reshape(probs, (5,5))

5×5 Matrix{Float64}:
 0.333333  1.0       0.952381  1.0       0.809524
 0.619048  0.619048  1.0       1.0       1.0
 0.142857  0.952381  0.714286  0.619048  1.0
 0.142857  1.0       1.0       0.952381  0.809524
 0.238095  0.952381  0.904762  0.47619   0.714286

In [118]:
sum(idxers)

25-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [112]:
curr_model = load_object("raw_model_0.jld2")
mps = predict_proba(curr_model, X)[:,2]
calced_probs = reshape(mps, (5,5))


pass_2_valid = (calced_probs .>= config.met_probs[1][1]) .& (calced_probs .<= config.met_probs[1][2])
pass_2_real = currs[config.mask_names[2]][:,:]

5×5 Matrix{Union{Missing, Float64}}:
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing   missing   missing
 1.0   missing  1.0       1.0        missing
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing  1.0       1.0

In [113]:
pass_2_real[BitMatrix(map(ismissing, pass_2_real))] .= 0

12-element view(reshape(::Matrix{Union{Missing, Float64}}, 25), [6, 8, 9, 11, 12, 14, 15, 16, 17, 19, 22, 23]) with eltype Union{Missing, Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [116]:
Matrix{Bool}(pass_2_real) == pass_2_valid

true

In [104]:
pass_2_real

5×5 Matrix{Union{Missing, Float64}}:
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing   missing   missing
 1.0   missing  1.0       1.0        missing
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing  1.0       1.0

In [98]:
pass_2_real

5×5 Matrix{Union{Missing, Float64}}:
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing   missing   missing
 1.0   missing  1.0       1.0        missing
 1.0   missing   missing   missing  1.0
 1.0  1.0        missing  1.0       1.0

In [89]:
idx = pass_2_valid .== 0 
pass_2_valid = Matrix{Union{Bool, Missing}}(pass_2_valid)

5×5 Matrix{Union{Missing, Bool}}:
 1  0  0  0  1
 1  1  0  0  0
 1  0  1  1  0
 1  0  0  0  1
 1  1  0  1  1

In [90]:
pass_2_valid[.! pass_2_valid] .= missing

12-element view(reshape(::Matrix{Union{Missing, Bool}}, 25), [6, 8, 9, 11, 12, 14, 15, 16, 17, 19, 22, 23]) with eltype Union{Missing, Bool}:
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing

In [92]:
pass_2_valid == pass_2_real

missing

In [75]:
idx

5×5 BitMatrix:
 0  1  1  1  0
 0  0  1  1  1
 0  1  0  0  1
 0  1  1  1  0
 0  0  1  0  0

In [74]:
pass_2_valid[idx] .== missing

12-element Vector{Missing}:
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing
 missing

In [67]:
pass_2_valid .== pass_2_real

5×5 Matrix{Union{Missing, Bool}}:
 1   missing   missing   missing  1
 1  1          missing   missing   missing
 1   missing  1         1          missing
 1   missing   missing   missing  1
 1  1          missing  1         1

In [61]:
currs["./toy_set.nc"]

LoadError: NetCDF error: [31mVariable './toy_set.nc' not found in file ./toy_set.nc[39m (NetCDF error code: -49)

In [57]:
###let's write a test here. 
currs = NCDataset("./toy_set.nc")
QCed_set = currs["VEL_QC"][:,:]

5×5 Matrix{Union{Missing, Float32}}:
 missing  -17.0       -11.0         1.0          missing
 missing     missing    7.0       -19.0       -15.0
 missing  -10.0          missing     missing   -6.0
 missing   -9.0       -14.0         6.0          missing
 missing   17.0       -15.0          missing     missing

In [143]:
sample_DBZ

5×5 Matrix{Union{Missing, Float32}}:
  8.0  28.0  35.0  57.0  65.0
 22.0  57.0  55.0  26.0  22.0
 33.0  19.0  16.0  54.0  13.0
 33.0   2.0  48.0  53.0  43.0
 43.0  29.0  10.0  54.0  63.0

In [249]:
##Set up a toy example to do the analytical calculations 

function slide_window(var::Matrix{Union{Missing, Float32}}, window::Matrix{Union{Missing,Float32}}, func; replace_missing = false)

    ##First, construct matrix and apply window 
    nrow, ncol = size(var)
    wrow, wcol = size(window) 

    ###int divide to get the windows off center 
    plusx = div(wcol, 2)
    plusy = div(wrow, 2)

    ###Get center coordinates of window 
    wc = (plusx+1, plusy+1)

    res = zeros(size(var))
    curr_view = fill(Missing, (size(window)))
    for i in 1:1:nrow
        for j in 1:1:ncol
            
            ###Calculate maximum and minimum indicies into the variable array 
            maxx = i + plusy > nrow ? nrow : i + plusy 
            maxy = j + plusx > ncol ? ncol : j + plusx
            minx = i - plusy < 1    ? 1    : i - plusy 
            miny = j - plusx < 1    ? 1    : j - plusx 
            

            ###Calculate indicies into the window/weight array 
            ###Difference here is that these will simply be in relation to the center of the window 
            windx = ((wc[1] - (i-minx)), (wc[1] + (maxx-i)))
            windy = ((wc[2] - (j-miny)), (wc[2] + (maxy-j)))

            ##Calculate resultant matrix    
            weighted = var[miny:maxy,minx:maxx] .* window[windy[1]:windy[2], windx[1]:windx[2]]
            #println(weighted)

            # ##i,j is the center index 
            # println([i,j])

            # printstyled("ROW WINDOW: $(minx : maxx) COL WINDOW: $(miny : maxy)\n", color=:green)
            # printstyled("WINDOW INDEX: $(windx) $(windy)\n", color=:blue)
            # if (i,j) == (5,4) 
            #     printstyled("ROW WINDOW: $(minx : maxx) COL WINDOW: $(miny : maxy)\n", color=:green)
            #     printstyled("WINDOW INDEX: $(windx) $(windy)\n", color=:blue)
            # end 

            res[j,i] = func(weighted)
        end 
    end 
    Matrix{Float32}(res)
end 

slide_window (generic function with 3 methods)

In [250]:
sample_DBZ[2:5, 1:5] .* sw[1:4, 1:5]

4×5 Matrix{Float32}:
 22.0  57.0  55.0  26.0  22.0
 33.0  19.0  16.0  54.0  13.0
 33.0   2.0  48.0  53.0  43.0
 43.0  29.0  10.0  54.0  63.0

In [255]:
sw

7×7 Matrix{Union{Missing, Float32}}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [268]:
@btime slide_window(sample_DBZ, sw, std)[:]

  6.942 μs (207 allocations: 14.83 KiB)


25-element Vector{Float32}:
 18.169113
 17.82045
 17.82045
 17.82045
 17.86197
 18.602419
 18.757486
 18.757486
 18.757486
 18.191597
 18.602419
 18.757486
 18.757486
 18.757486
 18.191597
 18.602419
 18.757486
 18.757486
 18.757486
 18.191597
 19.333801
 19.68295
 19.68295
 19.68295
 20.07818

In [88]:
open("./ok.txt", "w") do file 
    for task in tasks 
        write(file, " "  * task)
    end 
end 

In [269]:
sample_DBZ

5×5 Matrix{Union{Missing, Float32}}:
  8.0  28.0  35.0  57.0  65.0
 22.0  57.0  55.0  26.0  22.0
 33.0  19.0  16.0  54.0  13.0
 33.0   2.0  48.0  53.0  43.0
 43.0  29.0  10.0  54.0  63.0

In [14]:
####Writing tests for calculate_features 

function test_calculate_features()
    
    input_loc = ds_path
    tasks=["VEL", "DBZ", "STD(DBZ)", "STD(DBZ)"]
    weight_matrixes = [pw, pw, sw, rw]
    output_loc = joinpath(scratchspace, "trash_output.h5")
    HAS_INTERACTIVE_QC = true

    X1, Y1 = calculate_features(input_loc, tasks, weight_matrixes, output_loc, HAS_INTERACTIVE_QC;
            verbose=true, REMOVE_LOW_NCP=false, NCP_THRESHOLD=Float32(.2), QC_variable ="VG", remove_variable = "VEL" )


    @assert X1[:,2] == sample_DBZ[:]
    @assert X1[:,1] == sample_VEL[:]
    @assert Y1[:]   == .! map(ismissing, sample_VG[:])
    ###Test file-specified arguments version of calculate features 

    input_loc = ds_path 
    print(tasks)
    argfile_path = joinpath(scratchspace, "./sample_tasks.txt")
    open(argfile_path, "w") do file 
        for task in tasks 
            write(file, "," * task)
        end 
    end 

    X2, Y2 = calculate_features(input_loc, argfile_path, output_loc, true, QC_variable="VG", remove_variable="VEL", 
                        weight_matrixes = weight_matrixes)

    @assert X1 == X2
    @assert Y1 == Y2 

    
return X1, X2
    
end 

test_calculate_features (generic function with 1 method)

In [15]:
X1, X2 = test_calculate_features()

Processed /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/toy_set.nc in 0.10117197036743164 seconds
COMPLETED PROCESSING 1 FILES IN 0.1 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/trash_output.h5

WRITING DATA TO FILE OF SHAPE (25, 4)
X TYPE: Matrix{Float32}
["VEL", "DBZ", "STD(DBZ)", "STD(DBZ)"]COMPLETED PROCESSING 1 FILES IN 0.03 SECONDS
OUTPUTTING DATA IN HDF5 FORMAT TO FILE: /Users/ischluesche/.julia/scratchspaces/905eca60-9fa9-4fb3-8835-f5cd63a3719c/ronin_testing/trash_output.h5

WRITING DATA TO FILE OF SHAPE (25, 4)
X TYPE: Matrix{Float32}


(Float32[-2.0 27.0 22.49435 22.286674; 5.0 1.0 22.177752 22.827671; … ; -17.0 10.0 22.122326 23.450901; -17.0 16.0 21.28184 22.681032], Float32[-2.0 27.0 22.49435 22.286674; 5.0 1.0 22.177752 22.827671; … ; -17.0 10.0 22.122326 23.450901; -17.0 16.0 21.28184 22.681032])

In [29]:
Ronin._weighted_func

_weighted_func (generic function with 3 methods)

In [31]:
using ImageFiltering

In [80]:
function get_window_matrixes(x) 
    println("OK $(x[4,4])")
    x[4,4]
end 

get_window_matrixes (generic function with 1 method)

In [14]:
close(currs)

closed Dataset

In [15]:
###getting started way down here for developing the volume sweep QC 


gbs_path = "/Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/GROUND_BASED_SWEEPS/cfrad.20240828_235048.151_to_20240828_235612.131_SEAPOL_SUR.nc"
gbs = NCDataset(gbs_path)

abs_path = "/Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222204.187_to_20240705_222208.161_N42RF-TM_AIR.nc"
abs = NCDataset(abs_path)

Dataset: /Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/benchmark_NOAA_cfrads/cfrad.20240705_222204.187_to_20240705_222208.161_N42RF-TM_AIR.nc
Group: /

Dimensions
   time = 360
   range = 721
   sweep = 1
   string_length_8 = 8
   string_length_32 = 32
   status_xml_length = 1
   r_calib = 1
   frequency = 1

Variables
  volume_number  
    Attributes:
     long_name            = data_volume_index_number
     units                = 
     _FillValue           = -9999

  platform_type   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = platform_type
     options              = fixed, vehicle, ship, aircraft_fore, aircraft_aft, aircraft_tail, aircraft_belly, aircraft_roof, aircraft_nose, satellite_orbit, satellite_geostat

  primary_axis   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = primary_axis_of_rotation
     options              = axi

In [29]:
plot(gbs["elevation"][:])

In [13]:
currs["DBZ"]

DBZ (1197 × 3460)
  Datatype:    Union{Missing, Float32} (Float32)
  Dimensions:  range × time
  Attributes:
   _FillValue           = -32768.0
   units                = dBZ
   sampling_ratio       = 1.0
   grid_mapping         = grid_mapping
   coordinates          = time range


In [39]:
currs = NCDataset(gbs_path)

Dataset: /Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/GROUND_BASED_SWEEPS/cfrad.20240828_235048.151_to_20240828_235612.131_SEAPOL_SUR.nc
Group: /

Dimensions
   time = 3460
   range = 1197
   sweep = 14
   string_length = 32
   frequency = 1
   r_calib = 1

Variables
  time   (3460)
    Datatype:    Dates.DateTime (Float64)
    Dimensions:  time
    Attributes:
     long_name            = time in seconds since volume start
     units                = seconds since 2024-08-28T23:50:48Z
     standard_name        = time
     calendar             = gregorian
     comment              = times are relative to the volume start_time

  range   (1197)
    Datatype:    Float32 (Float32)
    Dimensions:  range
    Attributes:
     long_name            = Range from instrument to center of gate
     units                = meters
     spacing_is_constant  = true
     meters_to_center_of_first_gate = 400.0000059604645
     meters_between_gates = 100.00000149011612

  azimuth  

In [43]:
currs.dim["sweep"]

14

In [51]:
###Struct containing information about a given radar sweep 
###Useful for decomposing volume scans 

###In order to avoid overhead, we'll omit a file path and just do this for every file. 
Base.@kwdef struct Sweep
    ray_start_num::Int64
    ray_end_num::Int64 
end 



###Open the file, check the number of sweeps contained, and create a sweep object for each 

file_sweeps = NCDataset(gbs_path) do f0

    sweeps = [] 
    for nswp in 1:1:f0.dim["sweep"] 
        push!(sweeps, Sweep(1,1))

    end 
    sweeps

end 

14-element Vector{Any}:
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)

In [52]:
file_sweeps

14-element Vector{Any}:
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)
 Sweep(1, 1)

In [31]:
converted_outpath = "/Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/GROUND_BASED_SWEEPS/RADXOUT/20240828/cfrad.20240828_235048.151_to_20240828_235612.131_SEAPOL_SUR.nc"

"/Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/GROUND_BASED_SWEEPS/RADXOUT/20240828/cfrad.20240828_235048.151_to_20240828_235612.131_SEAPOL_SUR.nc"

In [32]:
currs_2 = NCDataset(converted_outpath)

Dataset: /Users/ischluesche/Documents/Grad_School/Research/Ronin/BENCHMARKING/GROUND_BASED_SWEEPS/RADXOUT/20240828/cfrad.20240828_235048.151_to_20240828_235612.131_SEAPOL_SUR.nc
Group: /

Dimensions
   time = 3460
   range = 1197
   sweep = 14
   string_length_8 = 8
   string_length_32 = 32
   status_xml_length = 1
   r_calib = 1
   frequency = 1

Variables
  volume_number  
    Attributes:
     long_name            = data_volume_index_number
     units                = 
     _FillValue           = -9999

  platform_type   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = platform_type
     options              = fixed, vehicle, ship, aircraft_fore, aircraft_aft, aircraft_tail, aircraft_belly, aircraft_roof, aircraft_nose, satellite_orbit, satellite_geostat

  primary_axis   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = primary_axis_of_rotation
     options   

In [35]:
plot(currs_2["azimuth"][:])